# Отбор признаков

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")


## 0. Данные

Источник: https://www.kaggle.com/mlg-ulb/creditcardfraud

The dataset contains transactions made by credit cards in September 2013 by European cardholders.
This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284,807 transactions. The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.

It contains only numerical input variables which are the result of a PCA transformation. Unfortunately, due to confidentiality issues, we cannot provide the original features and more background information about the data. Features V1, V2, … V28 are the principal components obtained with PCA, the only features which have not been transformed with PCA are 'Time' and 'Amount'. Feature 'Time' contains the seconds elapsed between each transaction and the first transaction in the dataset. The feature 'Amount' is the transaction Amount, this feature can be used for example-dependant cost-sensitive learning. Feature 'Class' is the response variable and it takes value 1 in case of fraud and 0 otherwise.

Given the class imbalance ratio, we recommend measuring the accuracy using the Area Under the Precision-Recall Curve (AUPRC). Confusion matrix accuracy is not meaningful for unbalanced classification.

In [ ]:
data = pd.read_csv("creditcard.csv")

In [ ]:
data.head()

**Задание** Давайте проверим, сколько "фрода" в этой выборке?

## 1. Поиск константных признаков

**Задание** Посчитайте стандартное отклонение признаков

## 2. Поиск коррелирующих признаков

**Задание**: составьте матрицу корреляций и визуализируйте ее. 
Проанализируйте, есть ли необходимость в удалении каких-либо признаков

## 3. Оценка T-score

In [ ]:
def get_t_score(column, target):
    data_ones = column[target == 1]
    data_zeros = column[target == 0]
    mean_dif = np.abs(data_ones.mean() - data_zeros.mean())
    stde_ones = data_ones.var() / data_ones.shape[0]
    stde_zeros = data_zeros.var() / data_zeros.shape[0]
    stre_total = (stde_ones + stde_zeros) ** (1 / 2)
    t_score = mean_dif / stre_total
    return t_score

**Задание**: Рассчитайте t_scores для всех признаков

**Задание**: визуализируйте важность признаков в виде столбцовой диаграммы

**Задание**: составьте список отфильтрованных признаков. Соберите в небольшую функцию eliminate_features с двумя аргументами: 
- features
- eliminated_features

Сохраните результат в переменной features_list_tscore

## 4. Отбор с помощью моделей

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

**Задание**: устроим здесь train_test_split со стратификацией по целевой переменной

In [ ]:
scaler = StandardScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=features)
X_test_scaled =  pd.DataFrame(scaler.transform(X_test), columns=features)

### Отбор с помощью линейных моделей

**Задание**: Обучим логистическую регрессию с аргументами 
- penalty='l1'
- solver='liblinear'

**Задание**: визуализируйте важность признаков в виде столбцовой диаграммы

**Задание**: составьте список самых важных фичей по версии логистической регрессии features_list_lr

### Отбор случайным лесом

**Задание**: Обучим случайный лес

**Задание**: визуализируйте важность признаков в виде столбцовой диаграммы

**Задание**: составьте список самых важных фичей по версии случайного леса features_list_rf

## 5. Будем учить на разных группах признаков

**Задание**: давайте соберем функцию для обучения. Подадим ей:
- model
- X_train
- X_test
- y_train
- y_test
- features: набор признаков для обучения
<br> 
В ответ она нам вернет: модель и значение roc_auc

**Задание**: а теперь примением ее для обучения логистической регрессии

**Задание**: а теперь примением ее для обучения случайного леса

## 6. t-SNE

- Cтохастическое вложение соседей с t-распределением (t-SNE)
- Метод, который использует поэтапный итерационный подход для низкоразмерного представления исходных данных с сохранением информации об их локальном соседстве.

<img src='https://miro.medium.com/max/4800/0*8t35MPHpbvy9WwDY'>

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(random_state=17, perplexity=15, n_jobs=-1)
tsne_representation = tsne.fit_transform(X_train_scaled)
plt.figure(figsize=(15, 15))
plt.scatter(tsne_representation[:, 0], tsne_representation[:, 1]);

**Задание**: обучим kmeans на обучающей выборке и посмотрим распределение кластеров на tsne-трансформации

## 7. PCA

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=2)
pca_result = pca.fit_transform(X_train_scaled)

In [ ]:
pca.explained_variance_ratio_

In [ ]:
plt.figure(figsize=(16,10))
sns.scatterplot(
    x=pca_result[:, 0], y=pca_result[:, 1],
    hue=y_train)

In [ ]:
plt.figure(figsize=(16,10))
sns.scatterplot(
    x=pca_result[:, 0], y=pca_result[:, 1],
    hue=k_means.labels_)